---

# Sequence to Sequence Models (COSC 6336)

<br>
<div style="text-align: right; font-size: large;"><i>authored by Gustavo Aguilar</i></div>
<div style="text-align: right; font-size: small;"><i>March 19, 2020</i></div>

---

In this notebook, we show a minimalistic implementation of sequence to sequence (seq2seq) models. We use LSTM-based encoder and decoder, and we optimize the model on a toy dataset from automatically-generated data. The high-level view of the model is shown in the image below:

<img src='images/encoder-decoder.png' width='70%'/>

Then, we improve our model using an attention-based decoder, as described in the [slides of the lecture](https://docs.google.com/presentation/d/1nmTSrwa-8Pi456rBYfTyIpx9Gz_Kc1WBpBOf0SC4vCI/edit?usp=sharing). 

Here's what we cover in this notebook:
1. Define a simple (auto-generated) dataset
2. Define the encoder 
3. Define the decoder 
4. Train the model
5. Evaluate and inspect the predictions
6. Compare with seq2seq with attention


Lastly, you will find the **assignment** at the end of the notebook.

---

In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook as tqdm

torch.manual_seed(1)
random.seed(1)

## 1. Define a simple (auto-generated) dataset

Our automatically-generated data will contain sequences with repeated and unordered letters. Such sequences must be mapped to alphabetically-sorted sequences of unique letters. Here are a few examples:
```
ccccaaabb        ->   abc
vvvrxduuu        ->   druvx
sddvvvzzuuuxxx   ->   dsuvxz
```

Note that this dataset will pose the problems mentioned during the lecture to a vanilla seq2seq model. 

In [2]:
def sorting_letters_dataset(size):
    dataset = []
    for _ in range(size):
        x = []
        for _ in range(random.randint(3, 10)):
            letter = chr(random.randint(97, 122))
            repeat = [letter] * random.randint(1, 3)
            x.extend(repeat)
        y = sorted(set(x))
        dataset.append((x, y))
    return zip(*dataset)

We will have two splits in our data, one for **training**, and another for validation. The training set will be **20,000** samples and we will use this set to update the parameters of the model. The **validation** set will be **5,000** samples, which we use to select the best model.

_**NOTE**: The validation set is never used to update the parameters of the model. Instead, we use it to make sure that the model is generalizing well, and not doing overfitting._

In [3]:
class Vocab:
    def __init__(self, vocab):
        self.itos = vocab
        self.stoi = {d:i for i, d in enumerate(self.itos)}
        
    def __len__(self):
        return len(self.itos) 
    
src_vocab = Vocab(['<pad>'] + [chr(i+97) for i in range(26)])
tgt_vocab = Vocab(['<pad>'] + [chr(i+97) for i in range(26)] + ['<start>', '<stop>'] )

train_inp, train_out = sorting_letters_dataset(20_000)
valid_inp, valid_out = sorting_letters_dataset(5_000)

Now, we need to map the text data into numeric values. These numeric values are indexes that correspond to the entries in the embedding lookup table. 

In [4]:
def map_elems(elems, mapper):
    return [mapper[elem] for elem in elems]

def map_many_elems(many_elems, mapper):
    return [map_elems(elems, mapper) for elems in many_elems]

train_x = map_many_elems(train_inp, src_vocab.stoi)
train_y = map_many_elems(train_out, tgt_vocab.stoi)

valid_x = map_many_elems(valid_inp, src_vocab.stoi)
valid_y = map_many_elems(valid_out, tgt_vocab.stoi)

## 2. Define the encoder

Our encoder will be a simple LSTM model with one layer and one direction. 

<img src='images/encoder.png' width='50%'/>

In [5]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, lstm_size, z_type, dropout=0.5):
        super(Encoder, self).__init__()
        self.z_index = z_type
        
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, lstm_size, batch_first=True)
        self.drop = nn.Dropout(dropout)
    
    def forward(self, inputs):
        device = next(self.parameters()).device
        
        seq = torch.tensor([inputs]).to(device) # (1, seqlen)
        emb = self.emb(seq) # (1, seqlen, emb_dim)
        emb = self.drop(emb) 
        
        outs, (h_n, c_n) = self.lstm(emb)
        
        if self.z_index == 1:
            return h_n[0], c_n[0] # (seqlen, lstm_dim)
        else:
            return outs # (1, seqlen, lstm_dim)

encoder = Encoder(vocab_size=len(src_vocab), emb_dim=64, lstm_size=128, z_type=1)
encoder

Encoder(
  (emb): Embedding(27, 64)
  (lstm): LSTM(64, 128, batch_first=True)
  (drop): Dropout(p=0.5, inplace=False)
)

## 3. Define de decoder

Similar to the encoder, the decoder will be a LSTM cell with one layer and one direction. 

<img src='images/decoder.png' width='50%'/>

In [6]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, lstm_size, dropout=0.5):
        super(Decoder, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTMCell(emb_dim, lstm_size)
        self.clf = nn.Linear(lstm_size, vocab_size)
        
        self.drop = nn.Dropout(dropout)
        self.objective = nn.CrossEntropyLoss(reduction="none")
        
    def forward(self, state, targets, curr_token, last_token):
        device = next(self.parameters()).device
        
        loss = 0
        shifted = targets + [last_token]
        for i in range(len(shifted)):
            inp = torch.tensor([curr_token]).to(device)
            
            emb = self.emb(inp)
            emb = self.drop(emb)
            
            state = self.lstm(emb, state)
            q_i, _ = state 
            q_i = self.drop(q_i)

            scores = self.clf(q_i)
            target = torch.tensor([shifted[i]]).to(device)
            loss += self.objective(scores, target)
            
            curr_token = shifted[i]
            
        return loss / len(shifted)

    def predict(self, state, curr_token, last_token, maxlen):
        device = next(self.parameters()).device
        preds = []
        for i in range(maxlen):
            inp = torch.tensor([curr_token]).to(device)
            emb = self.emb(inp)
            
            state = self.lstm(emb, state)
            h_i, _ = state
            
            scores = self.clf(h_i)
            pred = torch.argmax(torch.softmax(scores, dim=1))
            curr_token = pred
            
            if last_token == pred:
                break
            preds.append(pred)
        return preds
    
decoder = Decoder(vocab_size=len(tgt_vocab), emb_dim=64, lstm_size=128)
decoder

Decoder(
  (emb): Embedding(29, 64)
  (lstm): LSTMCell(64, 128)
  (clf): Linear(in_features=128, out_features=29, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
  (objective): CrossEntropyLoss()
)

### We also consider a decoder LSTM with attention

The attention version that we use for this implementation is Luong's attention, also known as multiplicative attention. 

Consider the encoder outputs $h = [h_1, h_2, \dots, h_n]$, and the query vector $q_j$ of the decoding time step $j$ as the hidden vector of the decoder LSTM, then we define multiplicative attention as follows:

$$
\begin{align}
u_i &= v^\intercal tanh(W [h_i + q_j]) \\
\alpha_i &= \frac{exp(u_i)}{\sum^N_k exp(u_k)} \\
c &= \sum^N_i \alpha_i h_i
\end{align}
$$

The implementation is as follows:

In [7]:
class Attention(nn.Module):
    def __init__(self, input_dim, attn_dim):
        super(Attention, self).__init__()
        self.W = nn.Linear(input_dim, attn_dim)
        self.v = nn.Linear(attn_dim, 1, bias=False)
        
    def forward(self, dec_hidden, enc_outs):
        # enc_outs -> (batch, seqlen, hidden)
        # dec_hidden -> (batch, hidden)
        
        seqlen = enc_outs.size(1)
        
        repeat_h = dec_hidden.unsqueeze(1)  # make room to repeat on seqlen dim
        repeat_h = repeat_h.repeat(1, seqlen, 1)  # (1, seqlen, hidden)

        concat_h = torch.cat((enc_outs, repeat_h), dim=2) # (1, seqlen, hidden*2)
        
        scores = self.v(torch.tanh(self.W(concat_h))) # (1, seqlen, 1)
        probs = torch.softmax(scores, dim=1)
        
        weighted = enc_outs * probs # (1, seqlen, hidden)
        
        context = torch.sum(weighted, dim=1, keepdim=False) # (1, hidden)
        combined = torch.cat((dec_hidden, context), dim=1)  # (1, hidden*2)
        
        return combined

Since we want the decoder to focus on the right hidden outputs of the encoder, we need the to modify the decoder:

In [8]:
class AttentionDecoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, lstm_size, attn_size):
        super(AttentionDecoder, self).__init__()
        
        self.lstm_size = lstm_size
        
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTMCell(emb_dim, lstm_size)
        self.attn = Attention(lstm_size * 2, attn_size)
        self.clf = nn.Linear(lstm_size * 2, vocab_size)
        
        self.drop = nn.Dropout(0.5)
        self.objective = nn.CrossEntropyLoss(reduction="none")
        
    def init_state(self, device):
        h_0 = torch.zeros(1, self.lstm_size).to(device)  # (batch, hidden_size)
        c_0 = torch.zeros(1, self.lstm_size).to(device)  # (batch, hidden_size)
        return h_0, c_0
        
    def forward(self, enc_outs, targets, curr_token, last_token):
        loss = 0
        device = enc_outs.device
        state = self.init_state(device)
        
        shifted = targets + [last_token]
        for i in range(len(shifted)):
            inp = torch.tensor([curr_token]).to(device) # (1,)
            
            emb = self.emb(inp) # (1, emb_dim)
            emb = self.drop(emb)
            
            state = self.lstm(emb, state)
            q_i, _ = state 
            q_i = self.drop(q_i) # (1, emb_dim)
            
            combined = self.attn(q_i, enc_outs)
            
            scores = self.clf(combined)
            target = torch.tensor([shifted[i]]).to(device)
            loss += self.objective(scores, target)
            
            curr_token = shifted[i]
            
        return loss / len(shifted)

    def predict(self, enc_outs, curr_token, last_token, maxlen):
        preds = []
        device = enc_outs.device
        state = self.init_state(device)
        
        for i in range(maxlen):
            inp = torch.tensor([curr_token]).to(device)
            emb = self.emb(inp)
            
            state = self.lstm(emb, state)
            q_i, _ = state
            
            combined = self.attn(q_i, enc_outs)
            
            scores = self.clf(combined)
            pred = torch.argmax(torch.softmax(scores, dim=1))
            curr_token = pred
            
            if last_token == pred:
                break
                
            preds.append(pred)
        return preds
    
decoder = AttentionDecoder(vocab_size=len(tgt_vocab), emb_dim=64, lstm_size=128, attn_size=100)
decoder 

AttentionDecoder(
  (emb): Embedding(29, 64)
  (lstm): LSTMCell(64, 128)
  (attn): Attention(
    (W): Linear(in_features=256, out_features=100, bias=True)
    (v): Linear(in_features=100, out_features=1, bias=False)
  )
  (clf): Linear(in_features=256, out_features=29, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
  (objective): CrossEntropyLoss()
)

## 4. Train the encoder-decoder model

During training, we pass the targets to the decoder so that it can be used as the ideal input at time step $i$, instead of using the decoder predictions of the previous time step $i-1$.

<img src='images/encoder-decoder-lstm.png' width='80%'/>

In [9]:
START_IX = tgt_vocab.stoi['<start>']
STOP_IX  = tgt_vocab.stoi['<stop>']

def shuffle(x, y):
    pack = list(zip(x, y))
    random.shuffle(pack)
    return zip(*pack)


def train(encoder, decoder, train_x, train_y, batch_size=50, epochs=10, print_every=1):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    encoder.to(device)
    decoder.to(device)

    enc_optim = optim.SGD(encoder.parameters(), lr=0.001, momentum=0.99)
    dec_optim = optim.SGD(decoder.parameters(), lr=0.001, momentum=0.99)

    encoder.train()
    decoder.train()

    for epoch in range(1, epochs+1):
        encoder.zero_grad(); enc_optim.zero_grad()
        decoder.zero_grad(); dec_optim.zero_grad()

        train_x, train_y = shuffle(train_x, train_y)

        epoch_loss = 0
        batch_loss = 0    
        
        for i in range(len(train_x)):
            x = train_x[i]
            y = train_y[i]
            
            batch_loss += decoder(encoder(x), y, START_IX, STOP_IX)

            if (i+1) % batch_size == 0:
                batch_loss.backward()
                enc_optim.step()
                dec_optim.step()

                encoder.zero_grad(); enc_optim.zero_grad()
                decoder.zero_grad(); dec_optim.zero_grad()

                epoch_loss += batch_loss.item()
                batch_loss = 0

        if epoch % print_every == 0:
            print(f"**** Epoch {epoch} - Loss: {epoch_loss / len(train_x):.6f} ****")
    return encoder, decoder

### Train the vanilla seq2seq model

In [11]:
encoder = Encoder(vocab_size=len(src_vocab), emb_dim=64, lstm_size=128, z_type=1)
decoder = Decoder(vocab_size=len(tgt_vocab), emb_dim=64, lstm_size=128)

print(encoder)
print(decoder)

encoder, decoder = train(encoder, decoder, train_x, train_y, batch_size=50, epochs=5, print_every=1)

Encoder(
  (emb): Embedding(27, 64)
  (lstm): LSTM(64, 128, batch_first=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Decoder(
  (emb): Embedding(29, 64)
  (lstm): LSTMCell(64, 128)
  (clf): Linear(in_features=128, out_features=29, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
  (objective): CrossEntropyLoss()
)
**** Epoch 1 - Loss: 1.991759 ****
**** Epoch 2 - Loss: 0.853610 ****
**** Epoch 3 - Loss: 0.456747 ****
**** Epoch 4 - Loss: 0.270997 ****
**** Epoch 5 - Loss: 0.174615 ****


In [12]:
torch.save({'encoder': encoder.state_dict(),
            'decoder': decoder.state_dict()}, 'seq2seq.pt')

## 5. Evaluate and inspect the model predictions

In [13]:
def predict(encoder, decoder, samples, index_to_elem):
    encoder.eval()
    decoder.eval()
    
    preds = []
    for i in range(len(samples)):
        pred = decoder.predict(encoder(samples[i]), START_IX, STOP_IX, maxlen=10)
        pred = [index_to_elem[ix] for ix in pred]
        preds.append(''.join(pred))

    return preds

In [14]:
predictions = predict(encoder, decoder, valid_x, tgt_vocab.itos)
groundtruth = [''.join(t) for t in valid_out]

accuracy_score(groundtruth, predictions)

0.978

In [15]:
for i in range(len(valid_inp[:20])):
    x = ''.join(valid_inp[i])
    y = ''.join(valid_out[i])
    
    print(f"{x} --> {y} --> {predictions[i]}")

jjuvvmmmauuu --> ajmuv --> ajmuv
ceeeppnnn --> cenp --> cenp
iiikkxxaaijjjnnaaeee --> aeijknx --> aeijknx
hqqtttnnnooawwc --> achnoqtw --> achnoqtw
zzooeeeooonnzkhjhhh --> ehjknoz --> ehjknoz
aaaepppiitmbbb --> abeimpt --> abeimpt
wssrriidyyyf --> dfirswy --> dfirswy
kkuhhcrrzzmmmwww --> chkmruwz --> chkmruwz
cssxx --> csx --> csx
ddooouulllelllc --> cdelou --> cdelou
lllkkkaaaggfihrrr --> afghiklr --> afghklr
ttgbbiillzocccuuu --> bcgilotuz --> bcgilotuz
lliiieeexiivvvtttrrrfffxx --> efilrtvx --> efilrtvx
parhhhylll --> ahlpry --> ahlpry
qqqccciialrrrvvvlzzz --> acilqrvz --> acilqrvz
cczppkq --> ckpqz --> ckpqz
ffffffsaa --> afs --> afs
kooopppwooeet --> ekoptw --> ekoptw
oyyyiiio --> ioy --> ioy
nnnyyxxuuufff --> fnuxy --> fnuxy


## 6. Compare with seq2seq with attention

In [16]:
encoder = Encoder(vocab_size=len(src_vocab), emb_dim=64, lstm_size=128, z_type=0)
decoder = AttentionDecoder(vocab_size=len(tgt_vocab), emb_dim=64, lstm_size=128, attn_size=100)

print(encoder)
print(decoder)

encoder, decoder = train(encoder, decoder, train_x, train_y, batch_size=50, epochs=5, print_every=1)

torch.save({'encoder': encoder.state_dict(),
            'decoder': decoder.state_dict()}, 'seq2seq+attn.pt')

Encoder(
  (emb): Embedding(27, 64)
  (lstm): LSTM(64, 128, batch_first=True)
  (drop): Dropout(p=0.5, inplace=False)
)
AttentionDecoder(
  (emb): Embedding(29, 64)
  (lstm): LSTMCell(64, 128)
  (attn): Attention(
    (W): Linear(in_features=256, out_features=100, bias=True)
    (v): Linear(in_features=100, out_features=1, bias=False)
  )
  (clf): Linear(in_features=256, out_features=29, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
  (objective): CrossEntropyLoss()
)
**** Epoch 1 - Loss: 1.433182 ****
**** Epoch 2 - Loss: 0.375678 ****
**** Epoch 3 - Loss: 0.209631 ****
**** Epoch 4 - Loss: 0.183923 ****
**** Epoch 5 - Loss: 0.147630 ****


In [17]:
predictions = predict(encoder, decoder, valid_x, tgt_vocab.itos)
groundtruth = [''.join(t) for t in valid_out]

accuracy_score(groundtruth, predictions)

0.9816

In [18]:
for i in range(len(valid_inp[:20])):
    x = ''.join(valid_inp[i])
    y = ''.join(valid_out[i])
    
    print(f"{x} --> {y} --> {predictions[i]}")

jjuvvmmmauuu --> ajmuv --> ajmuv
ceeeppnnn --> cenp --> cenp
iiikkxxaaijjjnnaaeee --> aeijknx --> aeijknx
hqqtttnnnooawwc --> achnoqtw --> achnoqtw
zzooeeeooonnzkhjhhh --> ehjknoz --> ehjknoz
aaaepppiitmbbb --> abeimpt --> abeimpt
wssrriidyyyf --> dfirswy --> dfirswy
kkuhhcrrzzmmmwww --> chkmruwz --> chkmruwz
cssxx --> csx --> csx
ddooouulllelllc --> cdelou --> cdelou
lllkkkaaaggfihrrr --> afghiklr --> afghiklr
ttgbbiillzocccuuu --> bcgilotuz --> bcgilotuz
lliiieeexiivvvtttrrrfffxx --> efilrtvx --> efilrtvx
parhhhylll --> ahlpry --> ahlpry
qqqccciialrrrvvvlzzz --> acilqrvz --> acilqrvz
cczppkq --> ckpqz --> ckpqz
ffffffsaa --> afs --> afs
kooopppwooeet --> ekoptw --> ekoptw
oyyyiiio --> ioy --> ioy
nnnyyxxuuufff --> fnuxy --> fnuxy


# Assignment

As you can see, the models are quite good at the task at hand. However, they are not perfect and you will see that the performance on the test data drops. One of the reasons is that the models were trained for a few epochs. Even though training for longer could improve the models, the code is not efficient because it does not use batches. Instead, we are iterating sample by sample without taking advantage of parallelization. Once you optimize this code, you will see that both models are good, and the attention brings a substantial boost in performance.

### Your task

Your task is to make this model efficient using batches. You can keep the same overall architecture, which is a single LSTM layer with one direction on both the encoder and decoder while also using attention. **As requirement**, you must use RNN-based models with attention, but the number of layers, directions, and hidden units of the RNN are up to you. Similarly, the attention method (either any version of Luong's attention or Bahdanau's attention) can be defined by you. You can also use self-attention if you prefer, but that is not required nor needed to achieve good results.


### Evaluation

We will measure the performance of your model using accuracy for the exact matches. Since this is a toy dataset, it's reasonable to measure the performance using this metric (it is worth mentioning that for tasks such as machine trasnlation, the official metric is BLEU). We will provide a test dataset (although auto-generated too) that is fixed for everyone to make all the systems comparable.

### Delivery

* The encoder and decoder models (a single binary file named `model.pt`)
* The solution notebook as HTML or PDF named `notebook.html` or `notebook.pdf`
    * displaying the epoch losses during training
    * showing the last cell in the notebook with the resulting accuracy on the test set. You should make this cell self-contained, meaning that it loads the test set, generates the predictions, and computes the accuracy.


### A few words on the solution

A possible solution of the assignment uses the exact same architectures (same LSTM and attention mechanisms and parameters), but it was trained on batches. An iteration over the entire training set (a.k.a. epoch) took around 12 seconds with 50 samples per batch, which is 400 backpropagation steps from the 20k training samples. We trained the model for 100 epochs (about 20 minutes on a GPU) and reached over 90% of accuracy on the test set (90.06% for seq2seq and 97.04% for seq2seq+attention). 

### The test data

The test data is generated in the same manner as the training/validation data, but with intentionally longer input sentences. Here's how the current models perform on the test set:

In [19]:
import pandas as pd

test_data = pd.read_csv('test.txt', delimiter='\t', header=None, usecols=[0,1])
test_inp, test_out = test_data[0], test_data[1]

test_inp[0], test_out[0]

('wwwttttjjjhhddmmmmqqffmmmdddcffu', 'cdfhjmqtuw')

In [20]:
test_x = map_many_elems(test_inp, src_vocab.stoi)
test_y = map_many_elems(test_out, tgt_vocab.stoi)

The vanilla Seq2Seq model

In [21]:
encoder = Encoder(vocab_size=len(src_vocab), emb_dim=64, lstm_size=128, z_type=1)
decoder = Decoder(vocab_size=len(tgt_vocab), emb_dim=64, lstm_size=128)

state_dict = torch.load('seq2seq.pt')
encoder.load_state_dict(state_dict['encoder'])
decoder.load_state_dict(state_dict['decoder'])

predictions = predict(encoder, decoder, test_x, tgt_vocab.itos)
groundtruth = [''.join(t) for t in test_out]

accuracy_score(groundtruth, predictions)

0.8784

The Seq2Seq + Attention model

In [22]:
encoder = Encoder(vocab_size=len(src_vocab), emb_dim=64, lstm_size=128, z_type=0)
decoder = AttentionDecoder(vocab_size=len(tgt_vocab), emb_dim=64, lstm_size=128, attn_size=100)

state_dict = torch.load('seq2seq+attn.pt')
encoder.load_state_dict(state_dict['encoder'])
decoder.load_state_dict(state_dict['decoder'])

predictions = predict(encoder, decoder, test_x, tgt_vocab.itos)
groundtruth = [''.join(t) for t in test_out]

accuracy_score(groundtruth, predictions)

0.899